In [1]:
#FINAL ANSWERS TO ALL THE QUESTIONS ASKED IN THE GOOGLE FORM
print("Match Winner: India is predicted to win.")
print("Margin of victory in terms of runs: 33 runs")
print("Margin of victory in terms of wickets: 3 wwickets")
print("Man of the Match: Virat Kohli")
print("Highest run scorer from India: Virat Kohli")
print("Highest run scorer from Australia: Glenn Maxwell")
print("Highest wicket taker from India: Mohammed Shami")
print("Highest wicket taker from Australia: Mitchell Starc")
print("Total runs prediction: 483 runs")
print("Total wickets prediction: 15")
print("Total number of sixes: 11")
print("Best bowling figures: Mohammed Shami")
print("First dismissal type: Caught[Caught by a fielder/wicket-keeper]")

Match Winner: India is predicted to win.
Margin of victory in terms of runs: 33 runs
Margin of victory in terms of wickets: 3 wwickets
Man of the Match: Virat Kohli
Highest run scorer from India: Virat Kohli
Highest run scorer from Australia: Glenn Maxwell
Highest wicket taker from India: Mohammed Shami
Highest wicket taker from Australia: Mitchell Starc
Total runs prediction: 483 runs
Total wickets prediction: 15
Total number of sixes: 11
Best bowling figures: Mohammed Shami
First dismissal type: Caught[Caught by a fielder/wicket-keeper]


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from datetime import datetime

# Loading Excel sheet into a pandas DataFrame
df = pd.read_excel("C:\\Users\\rakta\\OneDrive\\Desktop\\IndAusDataFinal.xlsx")

# Creating a binary column indicating whether the match involved India and Australia
df['India'] = df['Team'].apply(lambda x: 1 if 'India' in x else 0)
df['Australia'] = df['Team'].apply(lambda x: 1 if 'Australia' in x else 0)

# Creating a binary column indicating head-to-head matches
df['Head_to_Head'] = (df['India'] == 1) & (df['Australia'] == 1)

# Creating a binary column indicating matches played at the 'Ahmedabad' ground
df['Ahmedabad'] = df['Ground'].apply(lambda x: 1 if x == 'Ahmedabad' else 0)

# Converting 'Result' column to binary (1 for won, 0 for lost)
df['Result'] = df['Result'].apply(lambda x: 1 if x == 'won' else 0)

# Converting 'Start Date' to datetime and calculating days from today
df['Start Date'] = pd.to_datetime(df['Start Date'], format='%d-%b-%y')  # Adjust the date format accordingly
df['Days_from_Today'] = (datetime.now() - df['Start Date']).dt.days

# Define features and target variable
features = ['India', 'Australia', 'Head_to_Head', 'Ahmedabad', 'Days_from_Today']
target = 'Result'

# Splitting the dataset into training and testing sets
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_set[features], train_set[target])

# Extracting the best model from the grid search
best_model = grid_search.best_estimator_

# Making predictions on the test set
predictions = best_model.predict(test_set[features])

# Evaluation of the model
accuracy = accuracy_score(test_set[target], predictions)
print(f'Optimized Model Accuracy: {accuracy * 100:.2f}%')

# Creating a DataFrame for the match between India and Australia
match_data = pd.DataFrame({
    'India': [1],
    'Australia': [1],
    'Head_to_Head': [1],  # Assuming this is a head-to-head match
    'Ahmedabad': [1],  # Assuming the match is played in Ahmedabad
    'Days_from_Today': [(datetime.now() - datetime.strptime('2023-11-19', '%Y-%m-%d')).days],  # Change the date accordingly
})

# Prediction for the match
prediction = best_model.predict(match_data[features])

# Interpreting the prediction
if prediction[0] == 1:
    print("India is predicted to win.")
else:
    print("Australia is predicted to win.")

#Please note that it will take approximately 1 minute and 5 seconds for getting the result

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from datetime import datetime
from sklearn.metrics import accuracy_score
import re

# Loading Excel sheet into a pandas DataFrame
df = pd.read_excel("C:\\Users\\rakta\\OneDrive\\Desktop\\IndAusDataFinal.xlsx")

# Creating a binary column indicating whether the match involved India and Australia
df['India'] = df['Team'].apply(lambda x: 1 if 'India' in x else 0)
df['Australia'] = df['Team'].apply(lambda x: 1 if 'Australia' in x else 0)

# Creating a binary column indicating head-to-head matches
df['Head_to_Head'] = (df['India'] == 1) & (df['Australia'] == 1)

# Creating a binary column indicating matches played at the 'Ahmedabad' ground
df['Ahmedabad'] = df['Ground'].apply(lambda x: 1 if x == 'Ahmedabad' else 0)

# Converting 'Result' column to binary (1 for won, 0 for lost)
df['Result'] = df['Result'].apply(lambda x: 1 if x == 'won' else 0)

# Converting 'Start Date' to datetime and calculate days from today
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['Days_from_Today'] = (datetime.now() - df['Start Date']).dt.days

# Defining weights for each feature
weights = {'India': 1, 'Australia': 1, 'Head_to_Head': 2, 'Ahmedabad': 1, 'Days_from_Today': 0.1}
df['Weighted_Sum'] = sum(df[feature] * weights[feature] for feature in weights.keys())

# Defining features and target variable
features = ['India', 'Australia', 'Head_to_Head', 'Ahmedabad', 'Days_from_Today']
target = 'Result'

# Training a logistic regression model with weights
model = LogisticRegression(class_weight='balanced')
model.fit(df[features], df[target])

# Creating a DataFrame for the match between India and Australia
match_data = pd.DataFrame({
    'India': [1],
    'Australia': [1],
    'Head_to_Head': [1],  # Assuming this is a head-to-head match
    'Ahmedabad': [1],  # Assuming the match is played in Ahmedabad
    'Days_from_Today': [(datetime.now() - datetime.strptime('2023-11-19', '%Y-%m-%d')).days],  # Change the date accordingly
    # ASSUMING HERE IT IS THE 1ST INNINGS, IF WE WANT MARGIN OF VICTORY IN WICKETS, THEN TAKE 2ND INNINGS. 
    'Inns': [1],  
    'Margin_numeric': [0]  # Placeholder for margin prediction
})

# Making a prediction for the match result
prediction = model.predict(match_data[features])

# Prediction
if prediction[0] >= 0.5:
    print("India is predicted to win.")
else:
    print("Australia is predicted to win.")

# Preprocessing 'Margin' column
df['Margin_numeric'] = df['Margin'].apply(lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else 0)
df['Margin_numeric'] = df.apply(lambda row: -row['Margin_numeric'] if row['Inns'] == 2 else row['Margin_numeric'], axis=1)

# Training a decision tree regressor model for margin prediction
margin_features = ['India', 'Australia', 'Head_to_Head', 'Ahmedabad', 'Days_from_Today', 'Inns']
margin_target = 'Margin_numeric'
margin_model = DecisionTreeRegressor(random_state=1)  # Set random_state for reproducibility
margin_model.fit(df[margin_features].dropna(), df[margin_target].dropna())

# Making a prediction for the match margin
margin_prediction = margin_model.predict(match_data[margin_features])

if match_data['Inns'].iloc[0] == 2:
    print(f"Predicted Margin of Victory (wickets): {abs(int(margin_prediction[0]))}")
else:
    print(f"Predicted Margin of Victory (Runs): {margin_prediction[0]}")



India is predicted to win.
Predicted Margin of Victory (Runs): 33.0


C:\Users\rakta\AppData\Local\Temp\ipykernel_22500\2882879180.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start Date'] = pd.to_datetime(df['Start Date'])


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Loading the dataset from Excel
file_path = "C://Users//rakta//OneDrive//Desktop//PlayerFinalAus.xlsx"
data = pd.read_excel(file_path)

# Assigning weights for matches before and during the World Cup
weight_before_wc = 0.5
weight_during_wc = 2.0

# Calculating weighted batting average
data['Weighted Batting Avg'] = (
    weight_before_wc * data['Batting Avg before WC'] +
    weight_during_wc * data['Batting Avg during WC']
) / (weight_before_wc + weight_during_wc)

# Separating features (X) and target variables (y) for Weighted Batting Avg
X_avg = data[['Weighted Batting Avg', 'Matches', 'Country', 'Player Name']]
y_avg = data['Weighted Batting Avg']

# Separating features (X) and target variables (y) for Wickets
X_wickets = data[['Wickets', 'Matches', 'Bowl Avg', 'Country', 'Player Name']]
y_wickets = data['Wickets']

# Creating separate DataFrames for Indian and Australian players
indian_players = data[data['Country'] == 'India']
australian_players = data[data['Country'] == 'Australia']

# Creating separate models for Indian and Australian players for Batting Avg
model_avg_india = LinearRegression()
model_avg_australia = LinearRegression()

# Training models for Batting Avg
model_avg_india.fit(indian_players[['Weighted Batting Avg', 'Matches']], indian_players['Weighted Batting Avg'])
model_avg_australia.fit(australian_players[['Weighted Batting Avg', 'Matches']], australian_players['Weighted Batting Avg'])

# Predicting the weighted batting average for the entire dataset
y_pred_avg_india = model_avg_india.predict(X_avg[X_avg['Country'] == 'India'][['Weighted Batting Avg', 'Matches']])
y_pred_avg_australia = model_avg_australia.predict(X_avg[X_avg['Country'] == 'Australia'][['Weighted Batting Avg', 'Matches']])

# Extracting player names from 'Player Name' column
players_names_avg_india = X_avg[X_avg['Country'] == 'India']['Player Name']
players_names_avg_australia = X_avg[X_avg['Country'] == 'Australia']['Player Name']

# Creating DataFrames with predicted weighted batting averages and player names
predictions_df_avg_india = pd.DataFrame({'Player Name': players_names_avg_india, 'Predicted Weighted Batting Avg': y_pred_avg_india})
predictions_df_avg_australia = pd.DataFrame({'Player Name': players_names_avg_australia, 'Predicted Weighted Batting Avg': y_pred_avg_australia})

# Finding the player with the best predicted weighted batting average for India and Australia
best_batting_avg_player_india = predictions_df_avg_india.loc[predictions_df_avg_india['Predicted Weighted Batting Avg'].idxmax()]
best_batting_avg_player_australia = predictions_df_avg_australia.loc[predictions_df_avg_australia['Predicted Weighted Batting Avg'].idxmax()]

print(f'Highest Run Scorer for India: {best_batting_avg_player_india["Player Name"]} with '
      f'{best_batting_avg_player_india["Predicted Weighted Batting Avg"]:.2f}')
print(f'Highest Run Scorer for Australia: {best_batting_avg_player_australia["Player Name"]} with '
      f'{best_batting_avg_player_australia["Predicted Weighted Batting Avg"]:.2f}')

# Creating separate models for Indian and Australian players for Wickets
model_wickets_india = LinearRegression()
model_wickets_australia = LinearRegression()

# Training models for Wickets
model_wickets_india.fit(indian_players[['Wickets', 'Matches', 'Bowl Avg']], indian_players['Wickets'])
model_wickets_australia.fit(australian_players[['Wickets', 'Matches', 'Bowl Avg']], australian_players['Wickets'])

# Predicting the wickets for all players in India and Australia
y_pred_wickets_india = model_wickets_india.predict(X_wickets[X_wickets['Country'] == 'India'][['Wickets', 'Matches', 'Bowl Avg']])
y_pred_wickets_australia = model_wickets_australia.predict(X_wickets[X_wickets['Country'] == 'Australia'][['Wickets', 'Matches', 'Bowl Avg']])

# Extracting names from "Player name " feature
players_names_wickets_india = X_wickets[X_wickets['Country'] == 'India']['Player Name']
players_names_wickets_australia = X_wickets[X_wickets['Country'] == 'Australia']['Player Name']

# Creating DataFrames with predicted wickets and player names
predictions_df_wickets_india = pd.DataFrame({'Player Name': players_names_wickets_india, 'Predicted Wickets': y_pred_wickets_india})
predictions_df_wickets_australia = pd.DataFrame({'Player Name': players_names_wickets_australia, 'Predicted Wickets': y_pred_wickets_australia})

# Calculating the wickets-to-matches ratio for India and Australia
predictions_df_wickets_india['Wickets-to-Matches Ratio'] = predictions_df_wickets_india['Predicted Wickets'] / indian_players['Matches']
predictions_df_wickets_australia['Wickets-to-Matches Ratio'] = predictions_df_wickets_australia['Predicted Wickets'] / australian_players['Matches']

# Finding the player with the best wickets-to-matches ratio for India and Australia
best_wicket_taker_india = predictions_df_wickets_india.loc[predictions_df_wickets_india['Wickets-to-Matches Ratio'].idxmax()]
best_wicket_taker_australia = predictions_df_wickets_australia.loc[predictions_df_wickets_australia['Wickets-to-Matches Ratio'].idxmax()]

#Printing the final values
print(f'Highest Wicket Taker for India: {best_wicket_taker_india["Player Name"]} with '
      f'{best_wicket_taker_india["Wickets-to-Matches Ratio"]:.2%} wickets/match ratio')
print(f'Highest Wicket Taker for Australia: {best_wicket_taker_australia["Player Name"]} with '
      f'{best_wicket_taker_australia["Wickets-to-Matches Ratio"]:.2%} wickets/match ratio')
print(f'Best Bowling Figures in the match will be of: {best_wicket_taker_india["Player Name"]}')
print(f'Man of the Match will be: {best_batting_avg_player_india["Player Name"]}')

Highest Run Scorer for India: Virat Kohli with 91.48
Highest Run Scorer for Australia: Glenn Maxwell with 60.18
Highest Wicket Taker for India: Mohammed Shami with 195.92% wickets/match ratio
Highest Wicket Taker for Australia: Mitchell Starc with 190.38% wickets/match ratio
Best Bowling Figures in the match will be of: Mohammed Shami
Man of the Match will be: Virat Kohli


In [ ]:
import pandas as pd

# Reading the dataset from Excel Sheet
file_path = "C://Users//rakta//OneDrive//Desktop//narendramodistadium.xlsx"  # Use raw string to handle backslashes
df = pd.read_excel(file_path)

# Converting 'StartDate' to datetime format
df['StartDate'] = pd.to_datetime(df['StartDate'], format='%d-%b-%y')

# Assigning more weightage to World Cup matches
df['Weightage'] = df['StartDate'].apply(lambda x: 5 if x >= pd.to_datetime('2023-10-03') else 1)

# Calculating weighted averages for runs, wickets, and sixes
weighted_runs = (df['Weightage'] * df['Total Runs']).sum() / df['Weightage'].sum()
weighted_wickets = (df['Weightage'] * df['Total Wickets']).sum() / df['Weightage'].sum()
weighted_sixes = (df['Weightage'] * df['Sixes']).sum() / df['Weightage'].sum()

# Printing the calculated values
print(f'Total Runs: {weighted_runs}')
print(f'Total Wickets: {weighted_wickets}')
print(f'Total Sixes: {weighted_sixes}')

Total Runs: 482.95652173913044
Total Wickets: 14.91304347826087
Total Sixes: 11.173913043478262


In [ ]:
# For mode of dismissal, we are looking at the most common dismissal types in ODIs for Indian openers and Australian openers

#We have collected the data from howstat.com
# No. of times Shubman Gill has been caught:24
# No. of times Shubman Gill has been bowled: 7
# No. of times Shubman Gill has been declared LBW:4
# No. of times Shubman Gill has been run out:1
# MOST COMMON DIMISSAL TYPE FOR SHUBMAN GILL IS CATCH OUT 

# No. of times Rohit Sharma has been caught:102
# No. of times Rohit Sharma has been bowled: 39
# No. of times Rohit Sharma has been declared LBW:22
# No. of times Rohit Sharma has been run out:13
# MOST COMMON DIMISSAL TYPE FOR ROHIT SHARMA IS CATCH OUT

# No. of times David Warner has been caught:108
# No. of times David Warner has been bowled:25
# No. of times David Warner has been declared LBW:12
# No. of times David Warner has been run out:6
# MOST COMMON DIMISSAL TYPE FOR DAVID WARNER IS CATCH OUT

# No. of times Travis Head has been caught:36
# No. of times Travis Head has been bowled:12
# No. of times Travis Head has been declared LBW:4
# No. of times Travis Head has been run out:1
# MOST COMMON DIMISSAL TYPE FOR TRAVIS HEAD IS CATCH OUT

# No. of times Mitchell has been caught:49
# No. of times Mitchell has been bowled:13
# No. of times Mitchell has been declared LBW:5
# No. of times Mitchell has been run out:5
# MOST COMMON DIMISSAL TYPE FOR MITCHELL MARSH IS CATCH OUT


#Therefore, statistically, the first mode of dismissal is CATCH OUT.
print("The first mode of dismissal is CATCH OUT")



The first mode of dismissal is CATCH OUT
